In [ ]:
# in cm03 scanpy_1.9.3 environment
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker
sns.set(font="Arial", font_scale=1.15, style='ticks')
plt.rc("axes.spines", top=False, right=False)

In [ ]:
bamPEFragmentSize="/mnt/mone/Project/WC300/Tools/Anaconda3/envs/deeptools/bin/bamPEFragmentSize"
samtools="/mnt/mone/Project/WC300/Tools/Anaconda3/bin/samtools"
sed='/bin/sed'
#sed='/usr/bin/sed'

In [ ]:
os.system('f{bamPEFragmentSize} \
--bamfiles AK1_haplotagged_wIndels.bam \
--histogram AK1_HiFi_fragment_length_distribution.pdf \
--numberOfProcessors 55 \
--samplesLabel AK1_HiFi \
--plotTitle "Fragment Size of AK1 HiFi Data" \
--maxFragmentLength 0 \
--outRawFragmentLengths Fragment_Lengths_HiFi_Raw.tab \
--verbose')

In [ ]:
input_file="Fragment_Lengths_HiFi_Raw.tab"
output_file="Fragment_Lengths_HiFi_Raw_modified.tab"
line_count = os.popen(f'wc -l {input_file}').read().split()[0]

In [ ]:
os.system(f'{sed} -n 2,{line_count}p {input_file} > {output_file}')

In [ ]:
df = pd.read_table(f'{output_file}')
cumul_percentage = (df['Occurrences'].cumsum() / df['Occurrences'].sum()) * 100
df['Cumulative Percentage'] = cumul_percentage

In [ ]:
ax = sns.histplot(data=df, x="Size", weights="Occurrences", stat='percent', bins=100, kde=True, color="black")
ax.set_xlabel("Length of region mapped by HiFi reads (bp)")
ax.set_ylabel("Percentage (%)")
plt.tight_layout()

In [ ]:
ax = sns.histplot(data=df, x="Size", weights="Occurrences", stat='percent', bins=100, cumulative=True, element="poly", fill=False, color="black")
ax.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.axvline(x=18704, color='blue', linestyle='--')
ax.axhline(y=df[df['Size'] == 18704]['Cumulative Percentage'].values, color='blue', linestyle='--')
ax.text(x=18704-3000, y=ax.get_ylim()[1]+3, s=f'18,704bp ({np.round(df[df["Size"] == 18704]["Cumulative Percentage"].values, 2)[0]}%)', color='blue', size='x-small') # x-50 for plot aesthetics
ax.axvline(x=24722, color='red', linestyle='--')
ax.axhline(y=df[df['Size'] == 24722]['Cumulative Percentage'].values, color='red', linestyle='--')
ax.text(x=24722-1000, y=ax.get_ylim()[1]+1, s=f'24,722bp ({np.round(df[df["Size"] == 24722]["Cumulative Percentage"].values, 2)[0]}%)', color='red', size='x-small') # x-50 for plot aesthetics
ax.set_xlabel("Length of region mapped by HiFi reads (bp)")
ax.set_ylabel("Percentage (%)")
ax.set_title("Empirical CDF", y=1.05)
plt.tight_layout()

In [ ]:
# Mean Fragment Size
(df['Size'] * df['Occurrences']).sum() / df['Occurrences'].sum()